Copyright (c) 2023 Graphcore Ltd. All rights reserved.

# Benchmarking GNN message passing on IPU with PyTorch Geometric

One of the key features of GNNs is message passing where a node's features are passed to its neighbours through specific operations. Message passing essentially consists of two parts, namely gathering the node embeddings onto their outbound edges and scattering the result onto the nodes that these edges connect to. There can be a number of additional operations between the gather and scatter, depending on the architecture of the model you are interested in, and therefore it is useful to consider the gather and scatter operations as distinct steps in message passing.

![Message Passing](./static/gather_scatter.jpg "Message Passing")

In this notebook we will take a look at the performance of the gather and scatter operations on Graphcore hardware in order to understand the performance advantage IPUs offer for these operations. Accelerating these fundamental operations with the IPU also directly translates into performance advantages in message passing and GNNs as a whole.

In this notebook, we will:
 * Understand how to construct a harness to run benchmarks of particular operations on the IPU,
 * Run a sweep of scatter and gather operations on the IPU,
 * Take a look at the results of a larger sweep.

Let's get started!


### Running on Paperspace

The Paperspace environment lets you run this notebook with minimal effort. To improve your experience we preload datasets and preinstall packages, this can take a few minutes. If you experience errors immediately after starting a session, please try restarting the kernel before contacting support. If a problem persists or you want to give us feedback on the content of this notebook, please reach out to us through our community of developers using our [Slack channel](https://www.graphcore.ai/join-community) or raise a [GitHub issue](https://github.com/graphcore/examples).


In order to improve usability and support for future users, Graphcore would like to collect information about the
applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

- User progression through the notebook
- Notebook details: number of cells, code being run and the output of the cells
- Environment details

You can disable logging at any time by running `%unload_ext graphcore_cloud_tools.notebook_logging.gc_logger` from any cell.

To set up the requirements for running the session, simply run the following:

In [ ]:
%pip install -r requirements.txt
%load_ext graphcore_cloud_tools.notebook_logging.gc_logger

And for compatibility with the Paperspace environment variables we will do the following:

In [ ]:
import os
import poptorch

poptorch.setLogLevel("ERR")
executable_cache_dir = (
    os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/pyg-benchmark"
)

Now we are ready to benchmark the gather and scatter operations on the IPU.

## Setting up the benchmarks

In this section we will discuss how we will run the benchmarks in this notebook on the IPU.

First we create a simple module which runs a scatter operation. 

This module does the following:
 * Creates random input nodes and edge tensors of sizes `[num_inputs, num_features]` and `[num_outputs, num_inputs]` respectively,
 * Performs a `torch scatter` operation with the requested reduce type, using these inputs and returns the output.
 * Registers `buffers` for the inputs and outputs, which reduces the host-IPU communication.

See `benchmark_util.py` for the full implementation.

In [ ]:
from benchmark_util import ScatterOp

op = ScatterOp(num_inputs=4, num_features=4, num_outputs=16, reduce="sum")

print(f"{op.input = }")
print(f"{op.index = }")

In the context of GNNs you can think of the scatter operation as sending the node features (which have been collected from the source nodes using the gather operation) to the target nodes of each edge, and using the reduce operation where the same target node receives multiple features. In this way, the input size corresponds to the number of outbound edges times the feature vector size for each node in the graph, while the size of the indices corresponds to the number of neighbours for each node (with the actual values being up to the total number of nodes).

Now we wrap this module in a PopTorch `for_loop`. This creates a loop on the IPU that runs the benchmarked operation a specified number of times. We have put this in a separate module (`Benchmark`) which you can see in detail in `benchmark_util.py`.

If you are unfamiliar with PopTorch, you can get started with running your models on the IPU by following our [Introduction to PopTorch tutorial](https://github.com/graphcore/examples/tree/v3.2.0/tutorials/tutorials/pytorch/basics).

In [ ]:
from benchmark_util import Benchmark

model = Benchmark(num_repeats=10, operator=op)

This model is almost ready to run on IPUs but we still need to specify some PopTorch options we will use with the model. The most interesting ones are:
 * Turning on synthetic data - this disables host I/O to ensure we are measuring the performance of the operation only. See the [PopTorch documentation](https://docs.graphcore.ai/projects/poptorch-user-guide/en/latest/reference.html?highlight=enablesyntheticdata#poptorch.Options.enableSyntheticData) for more details. Feel free to try turning this option off and see how it changes the performance results.
 * Logging cycle count - this allows us to see the cycles of the operation. See the [PopTorch documentation](https://docs.graphcore.ai/projects/poptorch-user-guide/en/latest/reference.html?highlight=cycle%20count#poptorch.Options.logCycleCount) for more details.

In [ ]:
options = poptorch.Options()
options.enableSyntheticData(True)
options.logCycleCount(True)
options.enableExecutableCaching(executable_cache_dir)
options.connectionType(poptorch.ConnectionType.OnDemand)

We are now ready to convert our model into a PopTorch model and compile it, making it ready to run on IPUs:

In [ ]:
pop_model = poptorch.inferenceModel(model, options=options)
pop_model.compile()

We now create a `PerfMetrics` object to log the performance of our model. This tracks the number of IPU cycles for each call and will give us an indication of bandwidth of the underlying operation. For full details see the implementation in `benchmark_util.py`.

In [ ]:
from benchmark_util import PerfMetrics

metrics = PerfMetrics(pop_model, num_repeats=5)

Now we are ready to run a benchmark. We start by running a few warm up iterations to reduce measurement noise. Then we run the performance metrics in a loop and collect the benchmark results.

In [ ]:
def benchmark(pop_model, num_repeats):
    num_warmup_rounds = 5

    for _ in range(num_warmup_rounds):
        _ = pop_model()

    metrics = PerfMetrics(pop_model, num_repeats=num_repeats)
    benchmark_results = []

    for _ in range(num_repeats):
        _ = pop_model()
        values = metrics.update(pop_model.cycleCount())
        benchmark_results.append(values)

    return benchmark_results


benchmark_results = benchmark(pop_model, num_repeats=5)

And we can view the results:

In [ ]:
benchmark_results

Note that the effective bandwidth reported is based on the number of cycles and the performance of a Graphcore Bow-M2000.

We now have some performance data for our scatter operation. This is the approach we will use for a range of sizes of operations for both scatter and gather operations, and demonstrate the high performance of Graphcore IPUs while running these operations leading to message passing acceleration in GNNs.

## Running a sweep of *scatter* operations

We will apply the same approach that we used to run and benchmark performance of a single scatter operation to run a sweep across a range of data sizes. This will demonstrate the high performance of IPUs when running scatter operations.

First, let's decide on the parameters for our sweep and the range of values for each parameter. To save time running this notebook, we will only select a small sweep range. Feel free to extend the range to gather more performance data.

In [ ]:
num_inputs = [2**e for e in range(5, 7)]
num_features = [2**e for e in range(5, 7)]
num_outputs = [2**e for e in range(10, 11)]

Next, create a Cartesian product of these parameters:

In [ ]:
from itertools import product

sweep_params = product(num_inputs, num_features, num_outputs)

Finally, run everything we have discussed above using the parameter sweep, logging the results to a `pandas` dataframe.

In [ ]:
import pandas as pd

dfs = []
num_repeats = 128

for params in sweep_params:
    print(
        f"Benchmarking scatter with input size {params[0]}, "
        f"feature size {params[1]}, output size {params[2]}"
    )

    op = ScatterOp(
        num_inputs=params[0],
        num_features=params[1],
        num_outputs=params[2],
        reduce="sum",
    )

    model = Benchmark(num_repeats=10, operator=op)
    pop_model = poptorch.inferenceModel(model, options=options)
    pop_model.compile()

    benchmark_results = benchmark(pop_model, num_repeats=10)

    df = pd.DataFrame(benchmark_results)
    p = {
        k: [v] * len(df)
        for k, v in zip(("num_inputs", "num_features", "num_outputs"), params)
    }
    df = pd.concat([pd.DataFrame(p), df], axis=1)
    dfs.append(df.mean().to_frame().T)

scatter_dfs = pd.concat(dfs)

Now let's visualise the results:

In [ ]:
import seaborn as sns

sns.set_theme()

sns.pairplot(
    data=scatter_dfs,
    x_vars=["num_inputs", "num_outputs"],
    y_vars=["effective bandwidth (GiB/s)", "time (μs)"],
    hue="num_features",
)

As we have only run a small sweep in the interest of time, here is a plot for a larger sweep:

![Scatter Benchmarks](./static/scatter_benchmarks.png "Scatter Benchmarks")

These results show how well the IPU performs across a wide range of hyperparameters for the scatter operation. We can observe a significant decrease in computation time as sparsity increases (so for smaller input and output sizes), a situation in which GPUs provide negligible improvements ([Hosseini et al., 2022](https://arxiv.org/abs/2207.09955)).
So what is the performance gain with an IPU compared with an enterprise-grade GPU for example? The plots below show the performance gains for the size spectrum in which typical GNNs operate, as a speedup factor over the NVIDIA A100 (for completeness, we used <a href="https://github.com/graphcore-research/hydronet-gnn/blob/main/experiments/kernel_a100.ipynb" target="_blank">this notebook</a> to collect GPU data).

In [ ]:
from benchmark_util import quick_benchmarks_3d

quick_benchmarks_3d("scatter_add")

The figures highlight the advantage offered by the Graphcore IPU compared to an NVIDIA A100 (1x, blue plane). On the left we plot the entire performance data we have collected previously which includes a range of different feature sizes for each input-output pair as a scatter plot, while the figure on the right shows the average speedup across all feature sizes. We observe statistically significant speedups towards the lower range of tensor sizes, while the GPU approaches the IPU's performance as sizes increase. It is important to note that in the case of sparse access (small scatter input sizes) the IPU achieves over 16 times the performance of a GPU under the same conditions.

Ok, we have seen excellent scatter results. Now we can do the same with gather to visualise IPU performance on that operation as well.

## Running a sweep of *gather* operations

First we create a module wrapping the `index_select` operation. This is very similar to the scatter module but feel free to take a look at the entire implementation in `benchmark_util.py` for more details.

In [ ]:
from benchmark_util import GatherOp

op = GatherOp(num_inputs=4, num_features=4, num_outputs=16)

print(f"{op.input = }")
print(f"{op.index = }")

In the context of GNNs you can think of the gather operation as collecting the node features onto the connecting edges. And so the input size can be thought of as the number of nodes, the feature size the size of the node embeddings and the output size the number of edges.

Then, in a similar way to scatter, we can benchmark a sweep of different sized inputs and outputs to this operation. Again we have used the reduced the sweep range in the interest of time.

In [ ]:
import pandas as pd

sweep_params = product(num_inputs, num_features, num_outputs)
dfs = []
num_repeats = 128

for params in sweep_params:
    print(
        f"Benchmarking gather with input size {params[0]}, "
        f"feature size {params[1]}, output size {params[2]}"
    )

    op = GatherOp(num_inputs=params[0], num_features=params[1], num_outputs=params[2])

    model = Benchmark(num_repeats=10, operator=op)
    pop_model = poptorch.inferenceModel(model, options=options)
    pop_model.compile()

    benchmark_results = benchmark(pop_model, num_repeats=10)

    df = pd.DataFrame(benchmark_results)
    p = {
        k: [v] * len(df)
        for k, v in zip(("num_inputs", "num_features", "num_outputs"), params)
    }
    df = pd.concat([pd.DataFrame(p), df], axis=1)
    dfs.append(df.mean().to_frame().T)

gather_dfs = pd.concat(dfs)

And again lets visualise the results:

In [ ]:
import seaborn as sns

sns.set_theme()

sns.pairplot(
    data=gather_dfs,
    x_vars=["num_inputs", "num_outputs"],
    y_vars=["effective bandwidth (GiB/s)", "time (μs)"],
    hue="num_features",
)

Here is a larger sweep we have done previously in order to visualise the results:

![Gather Benchmarks](./static/gather_benchmarks.png "Gather Benchmarks")

Similarly to the scatter operation seen previously, these results show that sparse access is notably fast on the IPU with high performance in varying the number of inputs and outputs which GPUs do not tend to leverage ([Hosseini et al., 2022](https://arxiv.org/abs/2207.09955)).

Moreover, it is important to note here that these results are collected for one-dimensional gather, in order to report fair comparison results by maintaining a linear correspondance between memory accesses and the number of inputs and outputs, while multi-dimensional tensors could provide even more performance benefits compared to the GPU.

Let's look in closer detail on what this means in comparison to an enterprise-grade GPU:

In [ ]:
from benchmark_util import quick_benchmarks_3d

quick_benchmarks_3d("gather")

Et voilà! We again observe over 8x speedups compared to an A100 baseline (1x, blue plane), with significantly higher performance across a wide range of hyperparameters, including the typical ranges within which GNN computations reside. This further confirms the advantages of the IPU's unique hardware characteristics which translate into more efficient compute in the realm of message passing neural networks.

## Conclusion

We have successfully benchmarked gather and scatter operations on the IPU and seen signficant advantages when compared with GPUs. We also discussed these operations' relevance in message passing and thus GNNs as a whole, which helps shine a light on the relevance of these benchmark results. Concretely we have:

* Understood how to set up an operation benchmark on the IPU,
* Run a sweep of gather and scatter benchmarks on the IPU with various input and output size,
* Taken a look at the results and seen the great performance the IPU has doing gather and scatter operations.

## What next?

Now that you have seen the great performance IPUs offer for message passing, now why not get started writing your own PyTorch Geometric model for the IPU:

* Use our notebook for benchmarking gather scatter operations on the GPU to gather performance numbers to compare to,
* Take a look at some of our tutorials for getting started with PyTorch Geometric on the IPU, for example our `learning-pytorch-geometric-on-ipus/1_at_a_glance.ipynb` or our `learning-pytorch-geometric-on-ipus/2_a_worked_example.ipynb` tutorial,
* Run some of our examples, maybe try a molecular property prediction notebook like our example using SchNet: `graph-prediction/schnet-molecular-property-prediction/schnet_training.ipynb`.